In [29]:
from pypdf import PdfReader
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from dotenv import load_dotenv
from groq import Groq

In [30]:
load_dotenv()

True

In [2]:
reader = PdfReader('rag_test_document.pdf')

In [3]:
print(len(reader.pages))

3


In [4]:
extracted_text = ""

for i in range(len(reader.pages)):
    extracted_text+=reader.pages[i].extract_text() + '\n'

In [5]:
extracted_text

"The History & Science of Coffee\n A Comprehensive Guide for Curious Minds\nOrigins and Early History\nCoffee's story begins in the highlands of Ethiopia, where legend tells of a goat herder named Kaldi who\nnoticed his goats becoming unusually energetic after eating berries from a certain tree. Although this tale\nmay be more myth than fact, what is historically confirmed is that coffee plants (Coffea species) originated\nin the Kaffa region of Ethiopia. By the 9th century, coffee was being cultivated and traded across the\nArabian Peninsula, particularly in Yemen, where Sufi monks used it to stay alert during long hours of\nprayer.\nThe first coffeehouses — known as qahveh khaneh — appeared in Persia and later spread throughout\nthe Ottoman Empire during the 15th and 16th centuries. These establishments quickly became important\nsocial hubs where people gathered to share news, engage in conversation, and enjoy music. By the 17th\ncentury, coffee had made its way to Europe, initially 

In [6]:
len(extracted_text)

6403

In [7]:
cleaned_text = extracted_text.replace('\n',' ')
print(cleaned_text)

The History & Science of Coffee  A Comprehensive Guide for Curious Minds Origins and Early History Coffee's story begins in the highlands of Ethiopia, where legend tells of a goat herder named Kaldi who noticed his goats becoming unusually energetic after eating berries from a certain tree. Although this tale may be more myth than fact, what is historically confirmed is that coffee plants (Coffea species) originated in the Kaffa region of Ethiopia. By the 9th century, coffee was being cultivated and traded across the Arabian Peninsula, particularly in Yemen, where Sufi monks used it to stay alert during long hours of prayer. The first coffeehouses — known as qahveh khaneh — appeared in Persia and later spread throughout the Ottoman Empire during the 15th and 16th centuries. These establishments quickly became important social hubs where people gathered to share news, engage in conversation, and enjoy music. By the 17th century, coffee had made its way to Europe, initially met with susp

In [8]:
def chunk_text(text, chunk_size = 300, overlap = 50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start: end])
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(cleaned_text)
len(chunks)

26

In [9]:
chunks

["The History & Science of Coffee  A Comprehensive Guide for Curious Minds Origins and Early History Coffee's story begins in the highlands of Ethiopia, where legend tells of a goat herder named Kaldi who noticed his goats becoming unusually energetic after eating berries from a certain tree. Although",
 'after eating berries from a certain tree. Although this tale may be more myth than fact, what is historically confirmed is that coffee plants (Coffea species) originated in the Kaffa region of Ethiopia. By the 9th century, coffee was being cultivated and traded across the Arabian Peninsula, particul',
 ' and traded across the Arabian Peninsula, particularly in Yemen, where Sufi monks used it to stay alert during long hours of prayer. The first coffeehouses — known as qahveh khaneh — appeared in Persia and later spread throughout the Ottoman Empire during the 15th and 16th centuries. These establish',
 'uring the 15th and 16th centuries. These establishments quickly became important so

In [10]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = embedding_model.encode(chunks)
embeddings.shape

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1133.43it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


(26, 384)

In [11]:
embeddings = np.array(embeddings, dtype = np.float32)

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)

index.add(embeddings)

print(f"Total Vectors stored: {index.ntotal}")

Total Vectors stored: 26


In [12]:
query = "Where did coffee plants originally come from?"

query_emb = embedding_model.encode([query])

distances, indices = index.search(query_emb, k = 3)

for i, idx in enumerate(indices[0]):
    print(f"Result{i+1}")
    print(chunks[idx])

Result1
after eating berries from a certain tree. Although this tale may be more myth than fact, what is historically confirmed is that coffee plants (Coffea species) originated in the Kaffa region of Ethiopia. By the 9th century, coffee was being cultivated and traded across the Arabian Peninsula, particul
Result2
icion but soon embraced after Pope Clement VIII gave it his approval. ■ Quick Fact: The Dutch were the first Europeans to successfully cultivate coffee outside of Arabia, establishing plantations in their colonies in Java (Indonesia) in the early 1600s. The Global Spread Coffee's journey across the 
Result3
 named Baba Budan smuggled seven coffee beans into India in the 17th century, planting them in the hills of Karnataka. Today, Indian coffee — especially from regions like Nilgiris and Chikmagalur — is prized internationally for its unique flavor profiles.  — Page 2 of 3 — The Science Behind the Brew


In [13]:
print(indices)

[[1 4 7]]


In [22]:
context = ""

for idx in indices[0]:
    context += chunks[idx] + " "

In [32]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [33]:
prompt = f"""Answer the following question based only on the provided context. If the answer is not in the context, say "I don't have enough information to answer that."

Context: {context}

Question: {query}

Answer:"""

In [34]:
response = client.chat.completions.create(
    model = 'llama-3.1-8b-instant',
    messages=[
        {'role': "system","content": "You are helpful assistant that answers questions based on the provided context."},
        {'role': 'user', 'content': prompt}
    ],
    temperature=0.3,
    max_tokens=500
)

answer = response.choices[0].message.content
print(f'\n Answer: {answer}')


 Answer: Coffee plants (Coffea species) originally came from the Kaffa region of Ethiopia.
